In [1]:
from google.cloud import aiplatform

print(aiplatform.__version__)

1.43.0


In [2]:
from typing import Any, Callable, Optional, Tuple, Union

from google.cloud import bigquery
from vertexai.generative_models import (
    ChatSession,
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerationResponse,
    GenerativeModel,
    Part,
    Tool,
)

In [3]:
REGION = "us-central1"
PROJECT_ID='qwiklabs-asl-02-9dacbbe2194b'
DATASET_ID="project"

## チャットセッション内にfunction calling 実装

In [4]:
class ChatAgent:
    def __init__(
        self,
        model: GenerativeModel,
        tool_handler_fn: Callable[[str, dict], Any],
        max_iterative_calls: int = 5,
    ):
        self.tool_handler_fn = tool_handler_fn
        self.chat_session = model.start_chat()
        self.max_iterative_calls = 5

    def send_message(self, message: str) -> GenerationResponse:
        response = self.chat_session.send_message(message)

        # This is None if a function call was not triggered
        fn_call = response.candidates[0].content.parts[0].function_call

        num_calls = 0
        # Reasoning loop. If fn_call is None then we never enter this
        # and simply return the response
        while fn_call:
            if num_calls > self.max_iterative_calls:
                break

            # Handle the function call
            fn_call_response = self.tool_handler_fn(
                fn_call.name, dict(fn_call.args)
            )
            num_calls += 1

            # Send the function call result back to the model
            response = self.chat_session.send_message(
                Part.from_function_response(
                    name=fn_call.name,
                    response={
                        "content": fn_call_response,
                    },
                ),
            )

            # If the response is another function call then we want to
            # stay in the reasoning loop and keep calling functions.
            fn_call = response.candidates[0].content.parts[0].function_call

        return response

## 関数の定義

In [7]:
get_table_info_func = FunctionDeclaration(
    name="get_table_info",
    description="BigQuery テーブルとテーブルの説明を取得して、ユーザーの質問に適切に回答できるようにします。",
    parameters={
        "type": "object",
        "properties": {
            "table_id": {
                "type": "string",
                "description": "Fully qualified ID of BigQuery table",
            }
        },
    },
)

query_tool = Tool(
    function_declarations=[
        get_table_info_func,
    ],
)

In [8]:
client = bigquery.Client()
tables = client.list_tables(f"{PROJECT_ID}.{DATASET_ID}")
table_names = [table.table_id for table in tables]
print(table_names)

table_id_list=[]
for table_id in table_names:
    table_id_list.append(f"{PROJECT_ID}.{DATASET_ID}.{table_id}")
print(table_id_list)
# table_names = [table.table_id for table in tables]

# TABLE_ID_LIST=[]
# for table_id in table_names:
#     TABLE_ID_LIST.append(f"{PROJECT_ID}.{DATASET_ID}.{table_id}")                        

['CIF別取引属性_個人', 'CRM_交渉履歴', 'VISAデビット契約情報', 'チャネル別利用状況', '保険元帳', '名寄せ取引属性_日次', '完済融資バッチキー', '定期預金元帳', '為替取引明細', '融資バッチキー', '融資ローン元帳']
['qwiklabs-asl-02-9dacbbe2194b.project.CIF別取引属性_個人', 'qwiklabs-asl-02-9dacbbe2194b.project.CRM_交渉履歴', 'qwiklabs-asl-02-9dacbbe2194b.project.VISAデビット契約情報', 'qwiklabs-asl-02-9dacbbe2194b.project.チャネル別利用状況', 'qwiklabs-asl-02-9dacbbe2194b.project.保険元帳', 'qwiklabs-asl-02-9dacbbe2194b.project.名寄せ取引属性_日次', 'qwiklabs-asl-02-9dacbbe2194b.project.完済融資バッチキー', 'qwiklabs-asl-02-9dacbbe2194b.project.定期預金元帳', 'qwiklabs-asl-02-9dacbbe2194b.project.為替取引明細', 'qwiklabs-asl-02-9dacbbe2194b.project.融資バッチキー', 'qwiklabs-asl-02-9dacbbe2194b.project.融資ローン元帳']


In [9]:
def get_table_info():
    bq_client = bigquery.Client()
    list_table_info=[]
    for table_id in table_id_list:
        list_table_info.append(bq_client.get_table(table_id).to_api_repr())
    return list_table_info

In [10]:
get_table_info()

[{'kind': 'bigquery#table',
  'etag': 'yJXu2r1As2vX1aB8ynQyiA==',
  'id': 'qwiklabs-asl-02-9dacbbe2194b:project.CIF別取引属性_個人',
  'selfLink': 'https://bigquery.googleapis.com/bigquery/v2/projects/qwiklabs-asl-02-9dacbbe2194b/datasets/project/tables/CIF別取引属性_個人',
  'tableReference': {'projectId': 'qwiklabs-asl-02-9dacbbe2194b',
   'datasetId': 'project',
   'tableId': 'CIF別取引属性_個人'},
  'schema': {},
  'numBytes': '0',
  'numLongTermBytes': '0',
  'numRows': '0',
  'creationTime': '1729217034949',
  'lastModifiedTime': '1729217034995',
  'type': 'TABLE',
  'location': 'US',
  'numTotalLogicalBytes': '0',
  'numActiveLogicalBytes': '0',
  'numLongTermLogicalBytes': '0'},
 {'kind': 'bigquery#table',
  'etag': 'yTQzHzsONbv3vrQjF9X83w==',
  'id': 'qwiklabs-asl-02-9dacbbe2194b:project.CRM_交渉履歴',
  'selfLink': 'https://bigquery.googleapis.com/bigquery/v2/projects/qwiklabs-asl-02-9dacbbe2194b/datasets/project/tables/CRM_交渉履歴',
  'tableReference': {'projectId': 'qwiklabs-asl-02-9dacbbe2194b',
   '

In [11]:
def handle_query_fn_call(fn_name: str, fn_args: dict):
    """Handles query tool function calls."""
    print(f"Function calling: {fn_name} with args: {str(fn_args)}\n")
    
    if fn_name == "get_table_info":
        result = get_table_info()
    else:
        raise ValueError(f"Unknown function call: {fn_name}")
    
    return result
    

In [12]:
model = GenerativeModel(
    "gemini-1.0-pro-001",
    tools=[query_tool],
    generation_config=GenerationConfig(temperature=0.0),
)
chat = ChatAgent(model=model, tool_handler_fn=handle_query_fn_call)

In [13]:
# Insert an initialization prompt before the first chat to help guide model behavior and output style/format

init_prompt = """
    質問には簡潔でわかりやすい回答をお願いします。
BigQuery テーブルをクエリして得た情報のみを使用してください。
情報を捏造しないでください。
    
    Question:
"""

prompt = "融資についてのテーブルの説明をしてください"
response = chat.send_message(init_prompt + prompt)
print(response.text)

Function calling: get_table_info with args: {'table_id': 'bigquery-public-data.usa_names.usa_1910_2013'}

申し訳ありませんが、このテーブルには融資に関する説明がありません。


In [14]:
chat.chat_session.history

[role: "user"
 parts {
   text: "\n    \350\263\252\345\225\217\343\201\253\343\201\257\347\260\241\346\275\224\343\201\247\343\202\217\343\201\213\343\202\212\343\202\204\343\201\231\343\201\204\345\233\236\347\255\224\343\202\222\343\201\212\351\241\230\343\201\204\343\201\227\343\201\276\343\201\231\343\200\202\nBigQuery \343\203\206\343\203\274\343\203\226\343\203\253\343\202\222\343\202\257\343\202\250\343\203\252\343\201\227\343\201\246\345\276\227\343\201\237\346\203\205\345\240\261\343\201\256\343\201\277\343\202\222\344\275\277\347\224\250\343\201\227\343\201\246\343\201\217\343\201\240\343\201\225\343\201\204\343\200\202\n\346\203\205\345\240\261\343\202\222\346\215\217\351\200\240\343\201\227\343\201\252\343\201\204\343\201\247\343\201\217\343\201\240\343\201\225\343\201\204\343\200\202\n    \n    Question:\n\350\236\215\350\263\207\343\201\253\343\201\244\343\201\204\343\201\246\343\201\256\343\203\206\343\203\274\343\203\226\343\203\253\343\201\256\350\252\254\346\230\216\